# Summary

Try out openai api and test out a few utility functions for the project.

In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from enum import IntEnum, Enum
from functools import total_ordering
import matplotlib.pyplot as plt
from multipledispatch import dispatch
import numpy as np
import openai
import os
import pandas as pd
from pathlib import Path

from htools import *
from jabberwocky.config import C
from jabberwocky.utils import load_api_key, load_prompt, openai_auth, \
    bold, print_response

In [3]:
cd_root()

Current directory: /storage/jabberwocky


In [4]:
openai_auth()

In [6]:
C.engines[0]

'ada'

In [7]:
C.prices['curie'], C.prices[2]

(0.006, 0.006)

In [8]:
prompt = load_prompt('how_to')
print(prompt)

How to change a lightbulb:
1. Turn off the light switch. 
2. Using a cloth, push the bulb in, furn anti clockwise and remove it.
3. Insert the new bulb and twist to find the right fit. Push it in and turn clockwise.
4. Turn on the light to test it.

How to tie a tie:
1. Place tie around your neck.
2. Cross the wide end over the thinner end.
3. Run wide end under tie and pull it across again.
4. Pull the wide end through the center.
5. Loop through the knot.
6. Tighten the knot.

How to fry an egg:
1. Crack the Eggs. 
2. Add the Eggs to the Pan. 
3. Cover When the Edges Turn White.
4. Wait until it's done.
5. Serve.

How to


In [147]:
openai.api_key = load_api_key()

In [163]:
res = openai.Completion.create(engine=C.engines[0],
                               prompt=prompt, temperature=.4,
                               max_tokens=100)

In [196]:
text = res['choices'][0]['text']
print(text)

 make a cake:
1. Cut the Cake.
2. Roll it out.
3. Cut out the Design.
4. Cut out the Cake.
5. Roll it out again.
6. Cut out the design.
7. Roll it out again.

How to make a cake:
1. Cut the Cake.
2. Roll it out.
3. Cut out the Design.
4. Roll it out again.
5. Cut


In [197]:
res = openai.Completion.create(engine=C.engines[-1],
                               prompt=prompt, temperature=.4,
                               max_tokens=100)

In [198]:
text = res['choices'][0]['text']
print(text)

 make a cake:
1. Preheat the Oven.
2. Add the Ingredients in the Right Order.
3. Bake in the Oven.
4. Wait until it's done.
5. Serve.

How to make a pizza:
1. Preheat the Oven.
2. Add the Ingredients in the Right Order.
3. Bake in the Oven.
4. Wait until it's done.
5. Serve.




In [202]:
res_dates = openai.Completion.create(
    engine=C.engines[0],
    prompt=load_prompt('short_dates'),
    temperature=.4,
    max_tokens=30,
    logprobs=10
)

In [206]:
print(load_prompt('short_dates'))
print(res_dates['choices'][0]['text'])

Input: 3/1/20
Output: March 1, 2020

Input: 09-04-99
Output: September 4, 1999

Input: 11/01/2017
Output: November 1, 2017

Input: 04/11/21
Output:
 November 11, 2017

Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21



In [12]:
# Moved to test paperspace but don't want to save API credits.
prompt = load_prompt('short_dates')
res = """Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21"""
print_response(prompt, res)

Input: 3/1/20
Output: March 1, 2020

Input: 09-04-99
Output: September 4, 1999

Input: 11/01/2017
Output: November 1, 2017

Input: 04/11/21
Output:Input: 04/11/21
Output: November 11, 2017

Input: 04/11/21


## Scratch

Experimented with more complex solutions below but these were ridiculously over-complicated for my intended use case. Just trying stuff out for fun.

In [125]:
@total_ordering
@auto_repr
class Engine:
    
    def __init__(self, name, index, price):
        self.name = name
        self.index = index
        self.price = price
        
    def __eq__(self, other):
        return self.index == other.index
    
    def __lt__(self, other):
        return self.index < other.index
    
    def __str__(self):
        return self.name

In [126]:
class Engines:
    
#     i2eng = ['ada', 'babbage', 'curie', 'davinci']
#     eng2price = {'ada': .0008,
#                  'babbage': .0012,
#                  'curie': .006,
#                  'davinci': .06}
    
    i2eng = ['ada', 'babbage', 'curie', 'davinci']
    engines = [Engine(name, i, price) for i, (name, price) in 
               enumerate(zip(i2eng, [.0008, .0012, .006, .06]))]
    
    @dispatch(int)
    def __getitem__(self, i):
        return self.i2eng[i]
    
    @dispatch((list, tuple))
    def __getitem__(self, idx):
        return [self[i] for i in idx]
    
#     @dispatch(int)
#     def price(self, i):
#         return self.eng2price[self[i]]
    
#     @dispatch(str)
#     def price(self, name):
#         return self.eng2price[name]
    
#     def prices(self):
#         return self.eng2price

In [127]:
ENGINES = Engines()

In [128]:
ENGINES.engines

[Engine(name='ada', index=0, price=0.0008),
 Engine(name='babbage', index=1, price=0.0012),
 Engine(name='curie', index=2, price=0.006),
 Engine(name='davinci', index=3, price=0.06)]

In [129]:
ENGINES[0] < ENGINES[2]

True

In [130]:
ENGINES[[0, 2]]

['ada', 'curie']

In [131]:
ENGINES[0, 2]

['ada', 'curie']

In [132]:
for e in ENGINES:
    print(e)

ada
babbage
curie
davinci
